In [6]:
from PIL import Image

def remove_white_pixels(file_path):
    # Load the image
    image = Image.open(file_path)

    # Convert the image to RGBA (to support transparency)
    image = image.convert("RGBA")

    # Get the data of the image
    data = image.getdata()

    # Create a new data list, replacing white pixels with transparent pixels
    new_data = []
    for item in data:
        # Check if the pixel is white (255, 255, 255)
        if item[:3] == (255, 255, 255):
            # Replace white with transparent
            new_data.append((255, 255, 255, 0))
        else:
            new_data.append(item)

    # Update image data
    image.putdata(new_data)

    # Save the image
    image.save(file_path)

In [21]:
for frame_num in range (1, 34 + 1):
    remove_white_pixels(f'sparkle_frames/frame{frame_num}.png')

In [22]:
remove_white_pixels(f'kirby_frames/kirby.png')

In [23]:
remove_white_pixels(f'star_frames/star.png')

In [64]:
def shift(data, width, padding, wraparound=False):
    new_data = []
    data = list(data)
    for i in range(round(len(data) / width)):
        new_data += data[(1 + i*width):((i + 1) * width)]
        if wraparound:
            new_data.append(data[i * width])
        else:
            new_data.append(padding)
    return new_data

def generate_shifted_frames(first_frame_file_path, dimensions, save_path, n_frames, wraparound):
    height, width = dimensions
    assert (height == 32 and width == 64)
    image = Image.open(first_frame_file_path)
    image = image.convert("RGBA")
    data = image.getdata()
    image.save(f'{save_path}/frame1.png')
    for i in range(2, n_frames + 1):
        data = shift(data, width, padding=(255, 255, 255, 0), wraparound=wraparound)
        image.putdata(data)
        image.save(f'{save_path}/frame{i}.png')

def generate_sparkle_frames(first_frame_file_path, dimensions, save_path, n_frames):
    generate_shifted_frames(first_frame_file_path, dimensions, save_path, n_frames, False) #This could be True

def generate_night_sky_frames(first_frame_file_path, dimensions, save_path, n_frames):
    generate_shifted_frames(first_frame_file_path, dimensions, save_path, n_frames, True)

In [57]:
generate_sparkle_frames('layers/frame1.png', (32, 64), 'sparkle_frames_new',34)

In [70]:
generate_night_sky_frames('layers/night_sky.png', (32, 64), 'sky_frames', 63)

In [50]:
def opaque(pixel):
    r, g, b, _ = pixel
    return (r, g, b, 255)

def transparent(pixel):
    r, g, b, _ = pixel
    return (r, g, b, 0)

def layer_images(file_paths_with_processors, show = False):
    image = None
    for file_path_and_processor in file_paths_with_processors:
        file_path = None
        processor = lambda x : x
        if isinstance(file_path_and_processor, tuple):
            file_path, processor = file_path_and_processor
        else:
            file_path = file_path_and_processor
        layer = Image.open(file_path)
        layer = layer.convert("RGBA")
        layer = processor(layer)
        if image is None:
            image = layer
            continue
        layer_data = layer.getdata()

        image_data = image.getdata()
        new_image = []
        for layer_pix, img_pix in zip(layer_data, image_data):
            if img_pix[-1] == 0: #current image is transparent at this pixel
                new_image.append(layer_pix)
            else:
                new_image.append(img_pix)

        image.putdata(new_image)
    if show: 
        image.show()
    return image

In [51]:
def process_sparkles(image, dimensions, cutoff):
    height, width = dimensions
    assert (height == 32 and width == 64)
    data = image.getdata()
    new_data = []
    for i, pixel in enumerate(data):
        if i % width > cutoff or pixel[:3] == (0, 0, 0):
            new_data.append(transparent(pixel))
        else:
            new_data.append(pixel)
    image.putdata(new_data)
    return image
    

In [67]:
def get_kirby_frames(output_dir=None):
    final_frames = []
    for sparkle_frame_num in range(1, 34 + 1):
        kirby_layers_file_paths = [
            'kirby_frames/kirby.png', 
            'star_frames/star.png', 
            (
                f'sparkle_frames_new/frame{sparkle_frame_num}.png', 
                lambda image: process_sparkles(image, dimensions=(32, 64), cutoff=28)
            )
        ]
        final_frames.append(layer_images(kirby_layers_file_paths))
    if output_dir is not None:
        for i, frame in enumerate(final_frames):
            frame.save(f'{output_dir}/frame{i+1}.png')
    final_frames[0].save(
        'output.gif',
        save_all=True,
        append_images=final_frames[1:],  # Append the rest of the images
        duration=100,              # Duration between frames (in milliseconds)
        loop=0                     # Loop forever (use 1 for a single loop)
    )

In [68]:
get_kirby_frames('kirby_star_sparkles_frames')

In [69]:
import base64
from PIL import Image
from io import BytesIO

def base64_encode(file_path):
    # Load the image
    image = Image.open(file_path)

    # Convert the image to bytes
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    image_bytes = buffered.getvalue()

    # Encode the image bytes to base64
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')

    # Print or use the base64 encoded string
    return image_base64

In [59]:
output_string = ""
for i in range(1, 34 + 1):
    base64_encoding = base64_encode(f'kirby_star_sparkles_frames/frame{i}.png')
    output_string += f"FRAME{2 * i - 1} = base64.decode(\"{base64_encoding}\")\n"
    output_string += f"FRAME{2 * i} = base64.decode(\"{base64_encoding}\")\n"
with open('kirby_star_sparkles_frames/frames.star', 'w') as f:
    f.write(output_string)

In [66]:
output_string = ""
for i in range(1, 34 + 1):
    output_string += f'render.Image(src=FRAME{2 * i - 1}),\n'
    output_string += f'render.Image(src=FRAME{2 * i}),\n'
print(output_string)

render.Image(src=FRAME1),
render.Image(src=FRAME2),
render.Image(src=FRAME3),
render.Image(src=FRAME4),
render.Image(src=FRAME5),
render.Image(src=FRAME6),
render.Image(src=FRAME7),
render.Image(src=FRAME8),
render.Image(src=FRAME9),
render.Image(src=FRAME10),
render.Image(src=FRAME11),
render.Image(src=FRAME12),
render.Image(src=FRAME13),
render.Image(src=FRAME14),
render.Image(src=FRAME15),
render.Image(src=FRAME16),
render.Image(src=FRAME17),
render.Image(src=FRAME18),
render.Image(src=FRAME19),
render.Image(src=FRAME20),
render.Image(src=FRAME21),
render.Image(src=FRAME22),
render.Image(src=FRAME23),
render.Image(src=FRAME24),
render.Image(src=FRAME25),
render.Image(src=FRAME26),
render.Image(src=FRAME27),
render.Image(src=FRAME28),
render.Image(src=FRAME29),
render.Image(src=FRAME30),
render.Image(src=FRAME31),
render.Image(src=FRAME32),
render.Image(src=FRAME33),
render.Image(src=FRAME34),
render.Image(src=FRAME35),
render.Image(src=FRAME36),
render.Image(src=FRAME37),
render.Ima

In [77]:
kirby_dot_star = '''\
load("http.star", "http")
load("render.star", "render")
load("encoding/base64.star", "base64")

\
'''

n_kirby_frames = 34 #34 is arbitrary. Just where I decided to repeat the sparkles
kirby_slow_factor = 2
for i in range(1, n_kirby_frames + 1):
    base64_encoding = base64_encode(f'kirby_star_sparkles_frames/frame{i}.png')
    kirby_dot_star += f"KIRBY_FRAME{i} = base64.decode(\"{base64_encoding}\")\n"

kirby_dot_star += "\n"

n_sky_frames = 63 #63 is width - 1
sky_slow_factor = 20
for i in range(1, n_sky_frames + 1): 
    base64_encoding = base64_encode(f'sky_frames/frame{i}.png')
    kirby_dot_star += f"SKY_FRAME{i} = base64.decode(\"{base64_encoding}\")\n"


kirby_dot_star += '''\

def main():
    kirby_animation = render.Animation(
        children = [
\
'''

for i in range(1, n_kirby_frames + 1):
    for _ in range(kirby_slow_factor):
        kirby_dot_star += f'\t\t\trender.Image(src=KIRBY_FRAME{i}),\n'

kirby_dot_star += '''\
        ]
    )

    sky_animation = render.Animation(
        children = [
\
'''

for i in range(1, n_sky_frames + 1):
    for _ in range(sky_slow_factor):
        kirby_dot_star += f'\t\t\trender.Image(src=SKY_FRAME{i}),\n'

kirby_dot_star += '''\
        ]
    )

    return render.Root(
        child = render.Stack(children = [sky_animation, kirby_animation])
    )\
'''

with open('kirby.star', 'w') as f:
    f.write(kirby_dot_star)